In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMap
base_url = "https://api.waqi.info"
tok = '09314f421b346e8269181e8ccae540af098cdb02'
latlngbox = "8.0000,61.0000,37.0000,97.0000"
trail_url=f"/map/bounds/?latlng={latlngbox}&token={tok}"
my_data = pd.read_json(base_url + trail_url) 
print('columns->', my_data.columns) 


columns-> Index(['status', 'data'], dtype='object')


In [2]:
all_rows = []
for each_row in my_data['data']:
    all_rows.append([each_row['station']['name'],
                     each_row['lat'],
                     each_row['lon'],each_row['aqi']])
df = pd.DataFrame(all_rows,columns=['station_name', 'lat', 'lon','aqi'])

In [3]:
df['aqi'] = pd.to_numeric(df.aqi,errors='coerce') 
print('with NaN->', df.shape)

df1 = df.dropna(subset = ['aqi'])
print('without NaN->', df1.shape)

with NaN-> (203, 4)
without NaN-> (198, 4)


In [6]:
df2 = df1[['lat', 'lon', 'aqi']]
init_loc = [23, 77] # Approx over Bhopal
max_aqi = int(df1['aqi'].max())
print('max_aqi->', max_aqi)
m = folium.Map(location = init_loc, zoom_start = 5)
heat_aqi = HeatMap(df2, min_opacity = 0.1, max_val = max_aqi,
radius = 20, blur = 20, max_zoom = 2)
m.add_child(heat_aqi)

max_aqi-> 766


C:\Users\hp\AppData\Local\Temp\ipykernel_8528\657250229.py:6: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  heat_aqi = HeatMap(df2, min_opacity = 0.1, max_val = max_aqi,


In [7]:
centre_point = [23.25, 77.41] # Approx over Bhopal
m2 = folium.Map(location = centre_point,tiles = 'Stamen Terrain',zoom_start= 6)
for idx, row in df1.iterrows():
    lat = row['lat']
    lon = row['lon']
    station = row['station_name'] + ' AQI=' + str(row['aqi'])
    station_aqi = row['aqi']
    if station_aqi > 300: ## Red for very bad AQI
        pop_color = 'red'
    elif station_aqi > 200:
        pop_color = 'orange' ## Orange for moderate AQI
    else:
        pop_color = 'green' ## Green for good AQI
        
    folium.Marker(location= [lat, lon],popup = station,icon = folium.Icon(color = pop_color)).add_to(m2)
m2